In [12]:
import os
import re
import unicodedata
import pandas as pd
from collections import Counter
from rapidfuzz import fuzz, process
from rapidfuzz.distance import Levenshtein
from tqdm import tqdm

# ---------- CONFIG ----------
csv_file = "20250903_Extrait_Constatations_F2.csv"  # update path if needed
comment_col = "Commentaire"
type_col = "Type de constatation Texte"

# clustering params
min_words = 2
max_words = 5
comment_min_occurrences = 7   # minimum occurrences for Commentaire clusters
threshold_similarity = 90     # Rapidfuzz similarity %
max_typo_chars = 4            # Levenshtein distance tolerance
test_rows = None              # set e.g., 100 for small test
fuzzy_match_threshold = 85    # for row-wise correlation
checkpoint_every = 50         # save intermediate CSV every N clusters

# ---------- HELPERS ----------
def normalize_text(s):
    if pd.isna(s):
        return ""
    s = str(s)
    s = unicodedata.normalize("NFC", s)
    s = s.replace('\xa0', ' ')
    return s.strip()

def clean_words(text):
    if not text:
        return []
    text = normalize_text(text).lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    words = text.split()
    return [w for w in words if len(w) > 1]

def generate_clusters_from_words(words, min_len=min_words, max_len=max_words):
    clusters = []
    L = len(words)
    for n in range(min_len, min(max_len, L) + 1):
        for i in range(L - n + 1):
            clusters.append(' '.join(words[i:i+n]))
    return clusters

def join_variations_with_counts(variants, counter, delim=' | '):
    return delim.join([f"{v} ({counter.get(v,0)})" for v in variants])

def fuzzy_row_match(text, variants, threshold=85):
    text = str(text).lower()
    for v in variants:
        if fuzz.partial_ratio(text, v.lower()) >= threshold:
            return True
    return False

# ---------- 1. LOAD CSV ----------
df = pd.read_csv(csv_file, delimiter=';', dtype=str)
df.columns = [normalize_text(c) for c in df.columns]

if comment_col not in df.columns or type_col not in df.columns:
    raise KeyError(
        f"Expected columns '{comment_col}' and '{type_col}' not found. "
        f"Available columns: {df.columns.tolist()}"
    )

df[comment_col] = df[comment_col].astype(str)
df[type_col] = df[type_col].astype(str)

# optional test subset
if test_rows:
    df = df.head(test_rows)

# ---------- 2. EXTRACT COMMENTAIRE CLUSTERS ----------
all_comment_clusters = []
for text in tqdm(df[comment_col], desc="Extracting Commentaire clusters"):
    words = clean_words(text)
    clusters = generate_clusters_from_words(words, min_words, max_words)
    all_comment_clusters.extend(clusters)

cluster_counter = Counter(all_comment_clusters)
total_cluster_instances = sum(cluster_counter.values())
candidates = {k:v for k,v in cluster_counter.items() if v >= comment_min_occurrences}
print(f"Found {len(candidates)} unique Commentaire candidate clusters (≥{comment_min_occurrences} occurrences)")

# ---------- 3. FUZZY CLUSTERING ----------
cluster_groups = {}
for cluster in tqdm(candidates.keys(), desc="Fuzzy clustering Commentaire"):
    found = False
    for rep in list(cluster_groups.keys()):
        sim = fuzz.ratio(cluster, rep)
        edit_d = Levenshtein.distance(cluster, rep)
        if sim >= threshold_similarity or edit_d <= max_typo_chars:
            cluster_groups[rep].append(cluster)
            found = True
            break
    if not found:
        cluster_groups[cluster] = [cluster]

# ---------- 4. AGGREGATE COMMENTAIRE CLUSTERS ----------
comment_summary_rows = []
for rep, variants in cluster_groups.items():
    cnt = sum(cluster_counter.get(v,0) for v in variants)
    per_mille = round(cnt / total_cluster_instances * 1000, 2) if total_cluster_instances > 0 else 0
    variations_text = join_variations_with_counts(variants, cluster_counter, delim=' | ')
    comment_summary_rows.append({
        "Cluster_Representative": rep,
        "Count": cnt,
        "Variations": variations_text,
        "PerMille": per_mille
    })

comment_summary_df = pd.DataFrame(comment_summary_rows).sort_values(by="Count", ascending=False)
comment_summary_df.to_csv("commentaire_clusters_summary.csv", sep=';', index=False)
print("Saved 'commentaire_clusters_summary.csv' with", len(comment_summary_df), "rows.")

# ---------- 5. DEBUG: SAMPLE CLUSTERS ----------
print("\nSample clusters:")
for i,(rep, variants) in enumerate(list(cluster_groups.items())[:10]):
    print(f"{i+1}. Rep: {rep}, Variants: {variants}, Total Count: {sum(cluster_counter[v] for v in variants)}")

# ---------- 6. VECTORIZE ROW-WISE CORRELATION WITH CHECKPOINTS ----------
print("\nCorrelating Type de constatation Texte to Commentaire clusters (vectorized)...")
comment_texts = df[comment_col].str.lower().tolist()
type_values = df[type_col].tolist()
corr_rows = []

for i, (rep, variants) in enumerate(tqdm(cluster_groups.items(), desc="Vectorized correlation")):
    matches = process.cdist([rep], comment_texts, scorer=fuzz.partial_ratio)
    matched_indices = [j for j, score in enumerate(matches[0]) if score >= fuzzy_match_threshold]
    if not matched_indices:
        continue
    matched_types = [type_values[j] for j in matched_indices]
    type_counts = Counter(matched_types)
    total_type_count = sum(type_counts.values())
    for t_val, t_cnt in type_counts.items():
        t_per_mille = round(t_cnt / total_type_count * 1000, 2)
        corr_rows.append({
            "Commentaire_Cluster": rep,
            "Commentaire_Variations": join_variations_with_counts(variants, cluster_counter, delim=' | '),
            "Commentaire_Count": sum(cluster_counter.get(v,0) for v in variants),
            "Type_Value": t_val,
            "Type_Count": t_cnt,
            "Type_PerMille": t_per_mille
        })

    # === CHECKPOINT ===
    if (i+1) % checkpoint_every == 0:
        partial_df = pd.DataFrame(corr_rows)
        partial_df.to_csv(f"partial_corr_checkpoint_{i+1}.csv", sep=';', index=False)
        print(f"Checkpoint at cluster {i+1}: saved {len(partial_df)} rows.")
        print(partial_df.head(5))
        # Optional: stop after checkpoint for inspection
        # break

# ---------- 7. SAVE FINAL CORRELATIONS ----------
corr_df = pd.DataFrame(corr_rows)
if corr_df.empty:
    print("No correlations produced.")
    corr_df = pd.DataFrame(columns=[
        "Commentaire_Cluster",
        "Commentaire_Variations",
        "Commentaire_Count",
        "Type_Value",
        "Type_Count",
        "Type_PerMille"
    ])
corr_df.to_csv("commentaire_type_correlations.csv", sep=';', index=False)
print("Saved 'commentaire_type_correlations.csv' with", len(corr_df), "rows.")

# ---------- 8. DEBUG: SAMPLE CORRELATIONS ----------
print("\nSample correlations:")
print(corr_df.head(10))

# ---------- 9. ROW COVERAGE CHECK ----------
matched_rows_total = set()
for rep in cluster_groups.keys():
    matches = process.cdist([rep], comment_texts, scorer=fuzz.partial_ratio)
    matched_indices = [i for i, score in enumerate(matches[0]) if score >= fuzzy_match_threshold]
    matched_rows_total.update(matched_indices)
print(f"\nRows matched by clusters: {len(matched_rows_total)} out of {len(df)}")


Extracting Commentaire clusters: 100%|███████████████████████████████████████| 48227/48227 [00:00<00:00, 120178.51it/s]


Found 4671 unique Commentaire candidate clusters (≥7 occurrences)


Fuzzy clustering Commentaire: 100%|████████████████████████████████████████████████| 4671/4671 [02:19<00:00, 33.50it/s]


Saved 'commentaire_clusters_summary.csv' with 1898 rows.

Sample clusters:
1. Rep: stk bis, Variants: ['stk bis', 'stk bis 300', 'stk klein', 'stk bis 700', 'stk bis 100', 'stk bis 500', 'stk bis 75', 'stk bis 400', 'stk bis 900', 'stk gr', 'ft best', 'cm2 bis', 'stk bis 200', 'spr des', 'st tze', 'stk mit', 'stk bis 350', 'stk ca', 'nge bis', 'stk nge', 'stk block', 'stk bis 600', 'stk bis m2', 'je stk bis', 'stk bis 800', 'st ck', 'stk bis 150', 'st cke', 'stk riss', 'je bis', 'stk evtl', 'risse bis', 'ca bis', 'ist ein', '10 stk bis', 'stk bis 50', 'stk teils', 'ist in', 'stk risse', '10 bis', 'sse bis', 'oben bis', 'stk bis mm', 'stk bis ca', 'stk in', 'stk bis 450', 'von bis', 'cm bis', 'stk alte', 'ngen bis', '1stk bis', 'chig bis', 'stk pro', 'st be', 'stk bis 250', 'gt bis', 'st teils', '50 bis', 'ist die', 'stk je', 'riss bis', 'st sich', 'stk bis 750', 'lokal bis', 'stk gekr'], Total Count: 6517
2. Rep: bis 300, Variants: ['bis 300', 'bis 300 cm2', 'bis cm', 'bis 700', 'bis 1

Vectorized correlation:   3%|█▍                                                    | 50/1898 [03:58<1:25:12,  2.77s/it]

...

Checkpoint at cluster 1800: saved 15039 rows.
  Commentaire_Cluster                             Commentaire_Variations  \
0             stk bis  stk bis (2916) | stk bis 300 (261) | stk klein...   
1             stk bis  stk bis (2916) | stk bis 300 (261) | stk klein...   
2             stk bis  stk bis (2916) | stk bis 300 (261) | stk klein...   
3             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   
4             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   

   Commentaire_Count                                  Type_Value  Type_Count  \
0               6517  Fissures dues à des déformations empêchées           1   
1               6517                                      Eclats           1   
2               6517                             Nids de gravier           1   
3               5626                                      Eclats         263   
4               5626                          autre constatation           1   

   Type_PerMille

Vectorized correlation:  97%|██████████████████████████████████████████████████▋ | 1850/1898 [2:36:30<06:40,  8.34s/it]

Checkpoint at cluster 1850: saved 15237 rows.
  Commentaire_Cluster                             Commentaire_Variations  \
0             stk bis  stk bis (2916) | stk bis 300 (261) | stk klein...   
1             stk bis  stk bis (2916) | stk bis 300 (261) | stk klein...   
2             stk bis  stk bis (2916) | stk bis 300 (261) | stk klein...   
3             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   
4             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   

   Commentaire_Count                                  Type_Value  Type_Count  \
0               6517  Fissures dues à des déformations empêchées           1   
1               6517                                      Eclats           1   
2               6517                             Nids de gravier           1   
3               5626                                      Eclats         263   
4               5626                          autre constatation           1   

   Type_PerMille

Vectorized correlation: 100%|████████████████████████████████████████████████████| 1898/1898 [2:44:02<00:00,  5.19s/it]


Saved 'commentaire_type_correlations.csv' with 15494 rows.

Sample correlations:
  Commentaire_Cluster                             Commentaire_Variations  \
0             stk bis  stk bis (2916) | stk bis 300 (261) | stk klein...   
1             stk bis  stk bis (2916) | stk bis 300 (261) | stk klein...   
2             stk bis  stk bis (2916) | stk bis 300 (261) | stk klein...   
3             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   
4             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   
5             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   
6             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   
7             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   
8             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   
9             bis 300  bis 300 (318) | bis 300 cm2 (309) | bis cm (63...   

   Commentaire_Count                                  Type_Value  Type_Count  \
0 